In [2]:
import pandas as pd
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim
from unidecode import unidecode
import unidecode
import numpy as np
from shapely.geometry import Point, Polygon
import geopandas as gpd


import matplotlib.pyplot as plt

In [3]:
crime2023 = pd.read_excel('/Users/pablouriarte/Downloads/Municipal-Delitos-2015-2024_mar2024/2023.xlsx')


In [59]:
# Cambiar el formato de la columna 'Clave_Ent' a dos dígitos
crime2023['Clave_Ent'] = crime2023['Clave_Ent'].astype(str).apply(lambda x: x.zfill(2))

# Mostrar el DataFrame actualizado para verificar los cambios
print(crime2023['Clave_Ent'])

0         01
1         01
2         01
3         01
4         01
          ..
243329    32
243330    32
243331    32
243332    32
243333    32
Name: Clave_Ent, Length: 243334, dtype: object


In [66]:
mx_mun = gpd.read_file('/Users/pablouriarte/Mapa_Migracion_Irregular_Mexico_Fresh/1. Mapas/2. arcgis(general)/maps_boundaries/Municipalities/mapa_mexico_mun/Division_Municipal_Mexico_2010.shp').set_index('CLAVE').to_crs(epsg=4485)

In [68]:
# Crear la columna 'Cve. Municipio' concatenando 'CVE_EDO' y 'CVE_MUNI'
mx_mun['Cve. Municipio'] = mx_mun['CVE_EDO'].astype(str).str.zfill(2) + mx_mun['CVE_MUNI'].astype(str).str.zfill(3)

# Convertir la nueva columna a tipo entero para eliminar cualquier cero inicial no deseado
mx_mun['Cve. Municipio'] = mx_mun['Cve. Municipio'].astype(int)



In [92]:
mx_mun.info

<bound method DataFrame.info of                      NOM_MUN          NOMEDO CVE_EDO CVE_MUNI         Area  \
CLAVE                                                                        
01001         Aguascalientes  Aguascalientes      01      001  1168.762384   
01002               Asientos  Aguascalientes      01      002   547.762077   
01003               Calvillo  Aguascalientes      01      003   931.300088   
01004                  Cosio  Aguascalientes      01      004   128.907513   
01005            Jesus Maria  Aguascalientes      01      005   499.207990   
...                      ...             ...     ...      ...          ...   
32053  Villa Gonzalez Ortega       Zacatecas      32      053   417.529939   
32054          Villa Hidalgo       Zacatecas      32      054   355.528477   
32055             Villanueva       Zacatecas      32      055  2158.696446   
32056              Zacatecas       Zacatecas      32      056   444.547742   
32057               Trancoso    

In [94]:
crime2023.info

<bound method DataFrame.info of          Año Clave_Ent         Entidad  Cve. Municipio              Municipio  \
0       2023        01  Aguascalientes            1001         Aguascalientes   
1       2023        01  Aguascalientes            1001         Aguascalientes   
2       2023        01  Aguascalientes            1001         Aguascalientes   
3       2023        01  Aguascalientes            1001         Aguascalientes   
4       2023        01  Aguascalientes            1001         Aguascalientes   
...      ...       ...             ...             ...                    ...   
243329  2023        32       Zacatecas           32058  Santa María de la Paz   
243330  2023        32       Zacatecas           32058  Santa María de la Paz   
243331  2023        32       Zacatecas           32058  Santa María de la Paz   
243332  2023        32       Zacatecas           32058  Santa María de la Paz   
243333  2023        32       Zacatecas           32058  Santa María de la Paz

In [70]:
# Convertir 'Cve. Municipio' a entero
#mx_mun['Cve. Municipio'] = mx_mun['Cve. Municipio'].astype(int)

# Ordenar el DataFrame en forma ascendente por 'Cve. Municipio' como entero
mx_mun = mx_mun.sort_values(by='Cve. Municipio', ascending=True)

In [77]:
mx_mun

,NOM_MUN,NOMEDO,CVE_EDO,CVE_MUNI,Area,geometry,Cve. Municipio
CLAVE,,,,,,,
01001,Aguascalientes,Aguascalientes,01,001,1168.762384,"POLYGON ((1416489.577 2467700.472, 1417908.226...",1001
01002,Asientos,Aguascalientes,01,002,547.762077,"POLYGON ((1417043.958 2491681.240, 1417408.488...",1002
01003,Calvillo,Aguascalientes,01,003,931.300088,"POLYGON ((1347882.273 2454901.097, 1348002.307...",1003
01004,Cosio,Aguascalientes,01,004,128.907513,"POLYGON ((1397788.297 2509816.078, 1398009.089...",1004
01005,Jesus Maria,Aguascalientes,01,005,499.207990,"POLYGON ((1388272.165 2462097.533, 1389832.232...",1005
...,...,...,...,...,...,...,...
32053,Villa Gonzalez Ortega,Zacatecas,32,053,417.529939,"POLYGON ((1432967.088 2533295.874, 1432739.332...",32053
32054,Villa Hidalgo,Zacatecas,32,054,355.528477,"POLYGON ((1464010.774 2518715.060, 1464186.335...",32054
32055,Villanueva,Zacatecas,32,055,2158.696446,"POLYGON ((1345756.044 2531084.706, 1346503.579...",32055


In [78]:
import pandas as pd

# Assuming 'crime2023' is your DataFrame
# Display unique values in the 'Tipo de delito' column
unique_types = crime2023['Tipo de delito'].unique()
print("Unique values in 'Tipo de delito':")
print(unique_types)

# Display unique values in the 'Bien jurídico afectado' column
unique_subtypes = crime2023['Bien jurídico afectado'].unique()
print("\nUnique values in 'Bien jurídico afectado':")
print(unique_subtypes)


Unique values in 'Tipo de delito':
['Homicidio' 'Lesiones' 'Feminicidio' 'Aborto'
 'Otros delitos que atentan contra la vida y la integridad corporal'
 'Secuestro' 'Tráfico de menores' 'Rapto'
 'Otros delitos que atentan contra la libertad personal' 'Abuso sexual'
 'Acoso sexual' 'Hostigamiento sexual' 'Violación simple'
 'Violación equiparada' 'Incesto'
 'Otros delitos que atentan contra la libertad y la seguridad sexual'
 'Robo' 'Fraude' 'Abuso de confianza' 'Extorsión' 'Daño a la propiedad'
 'Despojo' 'Otros delitos contra el patrimonio' 'Violencia familiar'
 'Violencia de género en todas sus modalidades distinta a la violencia familiar'
 'Incumplimiento de obligaciones de asistencia familiar'
 'Otros delitos contra la familia' 'Corrupción de menores'
 'Trata de personas' 'Otros delitos contra la sociedad' 'Narcomenudeo'
 'Amenazas' 'Allanamiento de morada' 'Evasión de presos' 'Falsedad'
 'Falsificación' 'Contra el medio ambiente'
 'Delitos cometidos por servidores públicos' 'Electo

In [79]:
# Filtramos primero por las categorías principales de 'Bien jurídico afectado'
categorias_deseadas = [
    "La libertad y la seguridad sexual",
    "La sociedad",
    "La vida y la Integridad corporal",
    "Libertad personal",
    "Otros bienes jurídicos afectados (del fuero común)"
]

filtered_crime2023 = crime2023[crime2023['Bien jurídico afectado'].isin(categorias_deseadas)]

# Ahora definimos las subcategorías específicas para filtrar en 'Tipo de delito'
# según corresponda a la categoría principal
subcategorias_filtro = {
    "La vida y la Integridad corporal": ["Feminicidio"],
    "Libertad personal": ["Tráfico de menores"],
    "Otros bienes jurídicos afectados (del fuero común)": ["Narcomenudeo", "Delitos cometidos por servidores públicos"]
}

# Aplicar filtro adicional basado en subcategorías específicas
def filtro_adicional(row):
    if row['Bien jurídico afectado'] in subcategorias_filtro:
        # Si hay subcategorías específicas, filtrar por ellas
        return row['Tipo de delito'] in subcategorias_filtro[row['Bien jurídico afectado']]
    else:
        # Si no hay subcategorías específicas, mantener la fila
        return True

# Aplicamos el filtro adicional
filtered_crime2023 = filtered_crime2023[filtered_crime2023.apply(filtro_adicional, axis=1)]



In [80]:
# Eliminar la columna "Modalidad"
filtered_crime2023 = filtered_crime2023.drop('Modalidad', axis=1)

# Lista de nombres de las columnas de los meses
meses = [
    "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", 
    "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"
]

# Sumar las columnas de los meses para cada fila
filtered_crime2023['Total delitos'] = filtered_crime2023[meses].sum(axis=1)

# Eliminar las columnas individuales de los meses si ya no se necesitan
filtered_crime2023 = filtered_crime2023.drop(columns=meses)

In [81]:
filtered_crime2023

,Año,Clave_Ent,Entidad,Cve. Municipio,Municipio,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Total delitos
18,2023,01,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Feminicidio,Feminicidio,0
19,2023,01,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Feminicidio,Feminicidio,0
20,2023,01,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Feminicidio,Feminicidio,0
21,2023,01,Aguascalientes,1001,Aguascalientes,La vida y la Integridad corporal,Feminicidio,Feminicidio,0
29,2023,01,Aguascalientes,1001,Aguascalientes,Libertad personal,Tráfico de menores,Tráfico de menores,0
...,...,...,...,...,...,...,...,...,...
243321,2023,32,Zacatecas,32058,Santa María de la Paz,La sociedad,Corrupción de menores,Corrupción de menores,0
243322,2023,32,Zacatecas,32058,Santa María de la Paz,La sociedad,Trata de personas,Trata de personas,0
243323,2023,32,Zacatecas,32058,Santa María de la Paz,La sociedad,Otros delitos contra la sociedad,Otros delitos contra la sociedad,0
243324,2023,32,Zacatecas,32058,Santa María de la Paz,Otros bienes jurídicos afectados (del fuero co...,Narcomenudeo,Narcomenudeo,0


In [82]:
def etiquetar(row):
    if row['Bien jurídico afectado'] == 'La vida y la Integridad corporal' and row['Tipo de delito'] == 'Feminicidio':
        return 'Violencia Mujeres, Familias y Niños'
    elif row['Bien jurídico afectado'] == 'Libertad personal' and row['Tipo de delito'] == 'Tráfico de menores':
        return 'Violencia Mujeres, Familias y Niños'
    elif row['Bien jurídico afectado'] == 'La libertad y la seguridad sexual':
        return 'Violencia Mujeres, Familias y Niños'
    elif row['Bien jurídico afectado'] == 'La sociedad':
        return 'Violencia Mujeres, Familias y Niños'
    
    elif row['Bien jurídico afectado'] == 'Otros bienes jurídicos afectados (del fuero común)' and row['Tipo de delito'] == 'Narcomenudeo':
        return 'Crimen Organizado'
    
    elif row['Bien jurídico afectado'] == 'Otros bienes jurídicos afectados (del fuero común)' and row['Tipo de delito'] == 'Delitos cometidos por servidores públicos':
        return 'Corrupción Institucional'
    else:
        return 'Otra categoría'  # o puedes elegir un valor predeterminado más específico

# Aplicar la función para crear la nueva columna de etiquetas
filtered_crime2023['Etiqueta'] = filtered_crime2023.apply(etiquetar, axis=1)


In [83]:
# Eliminar las columnas que no se necesitan
columns_to_drop = ['Bien jurídico afectado', 'Tipo de delito', 'Subtipo de delito']
df_reduced = filtered_crime2023.drop(columns=columns_to_drop)

# Agrupar por 'Cve. Municipio' y 'Etiqueta', y sumar los valores de 'Total delitos'
df_aggregated = df_reduced.groupby(['Entidad','Municipio','Cve. Municipio', 'Etiqueta']).agg({'Total delitos': 'sum'}).reset_index()


In [95]:
df_aggregated.info

<bound method DataFrame.info of              Entidad       Municipio  Cve. Municipio  \
0     Aguascalientes  Aguascalientes            1001   
1     Aguascalientes  Aguascalientes            1001   
2     Aguascalientes  Aguascalientes            1001   
3     Aguascalientes        Asientos            1002   
4     Aguascalientes        Asientos            1002   
...              ...             ...             ...   
7444       Zacatecas      Villanueva           32055   
7445       Zacatecas      Villanueva           32055   
7446       Zacatecas       Zacatecas           32056   
7447       Zacatecas       Zacatecas           32056   
7448       Zacatecas       Zacatecas           32056   

                                 Etiqueta  Total delitos  
0                Corrupción Institucional            474  
1                       Crimen Organizado           1462  
2     Violencia Mujeres, Familias y Niños           1011  
3                Corrupción Institucional              4  


In [85]:
# Seleccionar las columnas necesarias de 'mx_mun'
mx_mun_reduced = mx_mun[['NOMEDO','Cve. Municipio', 'NOM_MUN', 'Area', 'geometry']]

# Convertir 'Cve. Municipio' en ambos DataFrames a tipo int para asegurar una comparación correcta
mx_mun_reduced['Cve. Municipio'] = mx_mun_reduced['Cve. Municipio'].astype(int)
df_aggregated['Cve. Municipio'] = df_aggregated['Cve. Municipio'].astype(int)

# Realizar el merge
df_final = pd.merge(df_aggregated, mx_mun_reduced, on='Cve. Municipio', how='left')


/Users/pablouriarte/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [86]:
# Resetear el índice del DataFrame para asegurar que no hayan duplicados
df_final = df_final.reset_index(drop=True)

In [91]:
df_final.info

<bound method DataFrame.info of              Entidad       Municipio  Cve. Municipio  \
0     Aguascalientes  Aguascalientes            1001   
1     Aguascalientes  Aguascalientes            1001   
2     Aguascalientes  Aguascalientes            1001   
3     Aguascalientes        Asientos            1002   
4     Aguascalientes        Asientos            1002   
...              ...             ...             ...   
7561       Zacatecas      Villanueva           32055   
7562       Zacatecas      Villanueva           32055   
7563       Zacatecas       Zacatecas           32056   
7564       Zacatecas       Zacatecas           32056   
7565       Zacatecas       Zacatecas           32056   

                                 Etiqueta  Total delitos          NOMEDO  \
0                Corrupción Institucional            474  Aguascalientes   
1                       Crimen Organizado           1462  Aguascalientes   
2     Violencia Mujeres, Familias y Niños           1011  Aguascali

In [88]:
# Intentar de nuevo filtrar para obtener solo las filas donde 'NOM_MUN' es NaN
nan_rows = df_final[df_final['NOM_MUN'].isna()]



In [89]:
nan_rows

,Entidad,Municipio,Cve. Municipio,Etiqueta,Total delitos,NOMEDO,NOM_MUN,Area,geometry
42,Baja California,San Felipe,2007,Corrupción Institucional,1,NaN,NaN,NaN,None
43,Baja California,San Felipe,2007,Crimen Organizado,22,NaN,NaN,NaN,None
44,Baja California,San Felipe,2007,"Violencia Mujeres, Familias y Niños",35,NaN,NaN,NaN,None
45,Baja California,San Quintín,2006,Corrupción Institucional,20,NaN,NaN,NaN,None
46,Baja California,San Quintín,2006,Crimen Organizado,58,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...
6871,Veracruz de Ignacio de la Llave,Santiago Sochiapan,30212,Crimen Organizado,0,NaN,NaN,NaN,None
6872,Veracruz de Ignacio de la Llave,Santiago Sochiapan,30212,"Violencia Mujeres, Familias y Niños",8,NaN,NaN,NaN,None
7512,Zacatecas,Santa María de la Paz,32058,Corrupción Institucional,0,NaN,NaN,NaN,None
7513,Zacatecas,Santa María de la Paz,32058,Crimen Organizado,0,NaN,NaN,NaN,None


In [90]:
# Eliminar todas las filas que tienen al menos un NaN en cualquier columna
df_final_clean = df_final.dropna()

# Mostrar el DataFrame limpio
print(df_final_clean)


             Entidad       Municipio  Cve. Municipio  \
0     Aguascalientes  Aguascalientes            1001   
1     Aguascalientes  Aguascalientes            1001   
2     Aguascalientes  Aguascalientes            1001   
3     Aguascalientes        Asientos            1002   
4     Aguascalientes        Asientos            1002   
...              ...             ...             ...   
7561       Zacatecas      Villanueva           32055   
7562       Zacatecas      Villanueva           32055   
7563       Zacatecas       Zacatecas           32056   
7564       Zacatecas       Zacatecas           32056   
7565       Zacatecas       Zacatecas           32056   

                                 Etiqueta  Total delitos          NOMEDO  \
0                Corrupción Institucional            474  Aguascalientes   
1                       Crimen Organizado           1462  Aguascalientes   
2     Violencia Mujeres, Familias y Niños           1011  Aguascalientes   
3                Corrup

In [102]:
# Definir la ruta donde deseas guardar el shapefile
shapefile_path = "/Users/pablouriarte/Mapa_Migracion_Irregular_Mexico_Fresh/1. Mapas/1. Mapas/Metodo_1/areas/df_final_shape"

# Guardar el GeoDataFrame como shapefile
gdf.to_file(f"{shapefile_path}.shp")

print("Shapefile guardado con éxito en:", shapefile_path + ".shp")


/var/folders/68/6zkv8z4j4rl6ds90br2rb7ph0000gn/T/ipykernel_7632/3149254785.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(f"{shapefile_path}.shp")


Shapefile guardado con éxito en: /Users/pablouriarte/Mapa_Migracion_Irregular_Mexico_Fresh/1. Mapas/1. Mapas/Metodo_1/areas/df_final_shape.shp


In [97]:
# Guardar el DataFrame final en un archivo CSV
path = "/Users/pablouriarte/Mapa_Migracion_Irregular_Mexico_Fresh/1. Mapas/1. Mapas/Metodo_1/areas/riskzone.csv"
df_final_clean.to_csv(path, index=False)


In [ ]:
import gzip
import shutil

# Ruta del archivo original
path_original = "/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/areas/riskzone.csv"
# Ruta del archivo comprimido
path_comprimido = "/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/areas/riskzone.csv.gz"

# Abrir el archivo original y el archivo comprimido
with open(path_original, 'rb') as f_in, gzip.open(path_comprimido, 'wb') as f_out:
    # Copiar el contenido del archivo original al archivo comprimido usando gzip
    shutil.copyfileobj(f_in, f_out)

print("Archivo comprimido con éxito.")
